In [1]:
import matplotlib.pyplot as plt, numpy as np, numpy.random as npr, csv, matplotlib.pyplot as plt, pandas as pd
import torch.nn as nn, timeit, torch, pystan as ps
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import LocallyLinearEmbedding
from tqdm import trange
%matplotlib inline
from time import time
from pylab import plot, show, legend

In [3]:
sm = ps.StanModel(file="/home/postdoc/dsen/Desktop/BNN_dimension_reduction/stan_files/nn_refpoints_constrainedtheta.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cf61588642e0539b092dedc55e934141 NOW.


In [4]:
data = pd.read_excel('concrete_data.xls').values
X = data.transpose()
Nobs = np.shape(X)[1]
print("Nobs =", Nobs, ", dim =", np.shape(X)[0])

Nobs = 1030 , dim = 9


In [15]:
n_ref = 60
lat_dim = 8
obs_dim = np.shape(X)[0] 
ref_points = npr.choice(Nobs,n_ref,replace=False)
embedding = LocallyLinearEmbedding(n_components=lat_dim, n_neighbors=10)
X_embedded = embedding.fit_transform(X.transpose()).transpose()

In [16]:
train = np.setdiff1d(np.arange(Nobs), ref_points)
X_ref = X[:,ref_points]
Z_ref = X_embedded[:,ref_points]
X_train = X[:,train]

In [17]:
n_in = np.shape(X_embedded)[0]
n_h = 20
n_out = np.shape(X)[0]

model = nn.Sequential(nn.Linear(n_in, n_h), nn.Tanh(), nn.Linear(n_h, n_out))

In [18]:
def init_normal(m):
    if type(m) == nn.Linear: nn.init.normal_(m.weight)
model.apply(init_normal)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)

In [19]:
n_iter = 1_000_000
Lss = np.zeros(n_iter)
start = time()
for epoch in range(n_iter):
    X_pred = model(torch.from_numpy(Z_ref.transpose().astype(np.float32)))
    loss = criterion(X_pred, torch.from_numpy(X_ref.transpose().astype(np.float32)))
    Lss[epoch] = loss.data
    if (epoch+1)%(n_iter/10) == 0 :
        print("Epoch: %d; loss: %.3f; time: %.1f mins" % (epoch+1, loss.item(), (time()-start)/60))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 100000; loss: 5009.125; time: 0.4 mins
Epoch: 200000; loss: 3879.376; time: 0.8 mins
Epoch: 300000; loss: 3301.369; time: 1.3 mins
Epoch: 400000; loss: 2272.964; time: 1.7 mins
Epoch: 500000; loss: 1999.415; time: 2.1 mins
Epoch: 600000; loss: 1957.538; time: 2.5 mins
Epoch: 700000; loss: 1936.182; time: 2.9 mins
Epoch: 800000; loss: 1930.975; time: 3.4 mins
Epoch: 900000; loss: 1894.366; time: 3.8 mins
Epoch: 1000000; loss: 1621.450; time: 4.2 mins
